<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [56]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [57]:
#derfinamos el url asociado a la libertad de prensa codigos por comodidad
url_codigo = 'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv'
# Primero, para a), hay que leer y concatenar los csv de ambos archivos, los cuales anteriormente vienen guardados en archivos_anio
dfs = [pd.read_csv(u) for u in archivos_anio]
df_anio = pd.concat(dfs, ignore_index=True)
#Ahora, tal como indica el inciso a, hay que normalizar a minusculas los nombres de columnas
df_anio.columns = df_anio.columns.str.lower()

#Para b), primero hay que revisar el archivo libertad_prensa_codigo.csv, el cual viene guardado anteriormente en df_codigos y luego identificar el codigo ISO
#y luego ver en que paises dicho codigo esta asociaod a dos paises distintos
df_codigos = pd.read_csv(url_codigo)
df_codigos.columns = df_codigos.columns.str.lower()
df_codigos['codigo_iso'] = df_codigos['codigo_iso'].astype(str).str.strip().str.upper()
df_codigos['pais'] = df_codigos['pais'].astype(str).str.strip()
repetidos = (df_codigos.groupby('codigo_iso')['pais']
             .nunique().reset_index(name='n')).query('n>1')['codigo_iso']

print("Códigos con dos nombres distintos:\n", repetidos.tolist())
print(df_codigos[df_codigos['codigo_iso'].isin(repetidos)].sort_values(['codigo_iso','pais']))

#Finalmente  para c), en primer lugar hay que asegurarse que la union conserve únicamente los registros que tengan coincidencia en ambas fuentes y posteriormente crear df
# de modo que combine df_anio con df_codigos
df_anio['codigo_iso'] = df_anio['codigo_iso'].astype(str).str.strip().str.upper()
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')
print("df shape:", df.shape)
print(df.head(3))




Códigos con dos nombres distintos:
 ['ZWE']
    codigo_iso      pais
179        ZWE  Zimbabue
180        ZWE      malo
df shape: (3077, 5)
  codigo_iso  anio  indice  ranking         pais
0        AFG  2001    35.5     59.0  Afghanistán
1        AGO  2001    30.2     50.0       Angola
2        ALB  2001     NaN      NaN      Albania




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [58]:
#Primero respondamos la estructura del DataFrame
print(f"Cantidad de filas del conjunto de datos: {df.shape[0]}")
print(f"Cantidad de columnas del DataFrame: {df.shape[1]}")
print("Nombres de las columnas del DataFrame:", list(df.columns))
print("\nTipos de datos de cada columna del DataFrame:\n", df.dtypes)
inesperado = [c for c in ['anio','indice','ranking'] if not pd.api.types.is_numeric_dtype(df[c])]
print("Las columnas con un tipo de dato inesperado son:", inesperado)
print("No hay columnas con un tipo de dato inesperado")
#Ahora para el resumen estadistico
print(df[['indice','ranking']].describe())

# mínimos y máximos (con país y año)
fila_min_ind = df.loc[df['indice'].idxmin(), ['pais','codigo_iso','anio','indice']]
fila_max_ind = df.loc[df['indice'].idxmax(), ['pais','codigo_iso','anio','indice']]
fila_min_rnk = df.loc[df['ranking'].idxmin(), ['pais','codigo_iso','anio','ranking']]
fila_max_rnk = df.loc[df['ranking'].idxmax(), ['pais','codigo_iso','anio','ranking']]
promedio = df['indice'].mean()
print("\nÍNDICE mínimo:", fila_min_ind.to_dict())
print("ÍNDICE máximo:", fila_max_ind.to_dict())
print("RANKING mínimo:", fila_min_rnk.to_dict())
print("RANKING máximo:", fila_max_rnk.to_dict())
print(f"Indice promedio: {promedio:.3f}")
print("Sobre los valores de indice y ranking, observo que en general los valores del minimo y maximo del indice son praticamente el doble entre si")

print("En la columna indice el valor minimo es 0.0, el maximo es 64536.0 y el promedio es 204.837")

print("Los países que presentan los valores extremos en indice y ranking son Dinamarca y Kosovo en el caso del indice, y Finlandia y Kosovo en el caso del ranking")

#Datos faltantes
print("Valores nulos en cada columna:\n", df.isna().sum())
prop_filas_con_nulos = (df.isna().any(axis=1).mean())
print(f"La proporción de observaciones que tienen valores faltantes es de: {prop_filas_con_nulos:.2%}")
null_pct = df.isna().mean().sort_values(ascending=False)
print("Columnas con mas del 30% de datos faltantes:\n", null_pct[null_pct > 0.30])
print("Basicamente ninguna columna tiene mas del 30% de datos faltantes")

#Unicidad y duplicados
print("Países distintos del DataFrame:", df['pais'].nunique())
print("Años distintos representados:", df['anio'].nunique())
dup_rows = df.duplicated().sum()
print("Filas duplicadas (exactamente iguales):", dup_rows)
print("No existen filas duplicadas")

#Finalmente, para la Validacion cruzada de columnas
validacion = (df.groupby('codigo_iso')['pais'].nunique().reset_index(name='n')).query('n>1')
print("\nInconsistencias entre el pais y su codigo ISO :", validacion)




Cantidad de filas del conjunto de datos: 3077
Cantidad de columnas del DataFrame: 5
Nombres de las columnas del DataFrame: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos de datos de cada columna del DataFrame:
 codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object
Las columnas con un tipo de dato inesperado son: []
No hay columnas con un tipo de dato inesperado
             indice        ranking
count   2680.000000    2854.000000
mean     204.836847     475.715137
std     2687.491660    6455.679267
min        0.000000       1.000000
25%       15.490000      35.000000
50%       28.150000      71.000000
75%       41.447500     110.000000
max    64536.000000  121056.000000

ÍNDICE mínimo: {'pais': 'Dinamarca', 'codigo_iso': 'DNK', 'anio': 2008, 'indice': 0.0}
ÍNDICE máximo: {'pais': 'Kosovo', 'codigo_iso': 'KSV', 'anio': 2014, 'indice': 64536.0}
RANKING mínimo: {'pais': 'Finlandia', 'codigo_iso': 'FIN', 'ani




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [59]:
## 1) Lista de países
america = ['ARG','ATG','BLZ','BOL','BRA','CAN','CHL','COL','CRI',
           'CUB','DOM','ECU','GRD','GTM','GUY','HND','HTI','JAM',
           'MEX','NIC','PAN','PER','PRY','SLV','SUR','TTO','URY',
           'USA','VEN']


df_america = df[df['codigo_iso'].isin(america)].copy()

years = sorted(df_america['anio'].unique())
res_for = []
for y in years:
    g = df_america.loc[(df_america['anio']==y) &
                       (df_america['indice'].notna()),
                       ['anio','pais','codigo_iso','indice']]
    if g.empty:
        continue
    i_min = g.loc[g['indice'].idxmin()]
    i_max = g.loc[g['indice'].idxmax()]
    res_for.append({
        'anio': y,
        'pais_indice_min': i_min['pais'], 'iso_min': i_min['codigo_iso'], 'indice_min': i_min['indice'],
        'pais_indice_max': i_max['pais'], 'iso_max': i_max['codigo_iso'], 'indice_max': i_max['indice'],
    })
res_for = pd.DataFrame(res_for).sort_values('anio').reset_index(drop=True)

# b) Realizamos lo mismo pero con un enfoque vectorizado con groupby, sin usar ciclos explícitos.
g = df_america[df_america['indice'].notna()].copy()
idx_min = g.groupby('anio')['indice'].idxmin()
idx_max = g.groupby('anio')['indice'].idxmax()

min_df = g.loc[idx_min, ['anio','pais','codigo_iso','indice']]\
          .rename(columns={'pais':'pais_indice_min','codigo_iso':'iso_min','indice':'indice_min'})
max_df = g.loc[idx_max, ['anio','pais','codigo_iso','indice']]\
          .rename(columns={'pais':'pais_indice_max','codigo_iso':'iso_max','indice':'indice_max'})

res_groupby = (min_df.merge(max_df, on='anio', how='outer')
                      .sort_values('anio')
                      .reset_index(drop=True))
display(res_for.head())
display(res_groupby.head())



,anio,pais_indice_min,iso_min,indice_min,pais_indice_max,iso_max,indice_max
0,2001,Canadá,CAN,0.8,Cuba,CUB,90.30
1,2002,Trinidad y Tobago,TTO,1.0,Cuba,CUB,97.83
2,2003,Trinidad y Tobago,TTO,2.0,Argentina,ARG,35826.00
3,2004,Trinidad y Tobago,TTO,2.0,Cuba,CUB,87.00
4,2005,Bolivia,BOL,4.5,Cuba,CUB,95.00


,anio,pais_indice_min,iso_min,indice_min,pais_indice_max,iso_max,indice_max
0,2001,Canadá,CAN,0.8,Cuba,CUB,90.30
1,2002,Trinidad y Tobago,TTO,1.0,Cuba,CUB,97.83
2,2003,Trinidad y Tobago,TTO,2.0,Argentina,ARG,35826.00
3,2004,Trinidad y Tobago,TTO,2.0,Cuba,CUB,87.00
4,2005,Bolivia,BOL,4.5,Cuba,CUB,95.00


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [60]:
#primero construimos la tabla pivot especificficada
pivot = pd.pivot_table(
    df, index='pais', columns='anio', values='indice',
    aggfunc='max', fill_value=0
).sort_index()

print("pivot shape:", pivot.shape)
print(pivot.head(5))

#Ahora, para poder obtener lo requerido de la tabla anterior, primero para a)
apilado = pivot.stack()
indice_max = apilado.idxmax()
valor_max = apilado.loc[indice_max]
apilado_nz = apilado[apilado > 0]
indice_min_nz = apilado_nz.idxmin()
valor_min_nz = apilado_nz.loc[idx_min_nz]

print(f"Máximo índice: {valor_max:.3f} | País: {indice_max[0]} | Año: {indice_max[1]}")
print(f"Mínimo índice (≠0): {valor_min_nz:.3f} | País: {indice_min_nz[0]} | Año: {indice_min_nz[1]}")

#Para obtener los años que presentan en promedio los valores de indices mas altos y mas bajos tenemos que
prom_anual = pivot.mean(axis=0)
año_max = prom_anual.idxmax()
año_min = prom_anual.idxmin()
print(f"Año con el promedio de indice más alto es: {año_max} | {prom_anual.loc[año_max]:.3f}")
print(f"Año con promedio de indice más bajo es: {año_min} | {prom_anual.loc[año_min]:.3f}")

#Para obtener el pais con mayor variabilidad
piv_nan = pivot.replace(0, np.nan)
variabilidad = piv_nan.max(axis=1) - piv_nan.min(axis=1)
pais_varmax = variabilidad.idxmax()
print(f"El pais con mayor variabilidad es: {pais_varmax} | rango: {variabilidad.loc[pais_varmax]:.3f}")

#d)
nuniq_no0 = piv_nan.apply(lambda s: s.dropna().nunique(), axis=1)
constantes = nuniq_no0[nuniq_no0 == 1].index.tolist()
print("Países con índice constante en los años regsistrados:", constantes)

#Finalmente determinar que paises no tienen ningun dato y explicar el porque
sin_datos = pivot.index[(pivot.sum(axis=1) == 0)].tolist()
print("Países sin ningún dato:", sin_datos)
print("Se concluye que no hay ningun pais que no tenga ningun dato. Esto se debe a que el pivot reemplaza automaticamente las celdas que no posean datos por 0.")






pivot shape: (180, 16)
anio         2001   2002   2003   2004   2005   2006   2007   2008   2009  \
pais                                                                        
Afghanistán  35.5  40.17  28.25  39.17  44.25  56.50  59.25  54.25  51.67   
Albania       0.0   6.50  11.50  14.17  18.00  25.50  16.00  21.75  21.50   
Alemania      1.5   1.33   2.00   4.00   5.50   5.75   4.50   3.50   4.25   
Algeria      31.0  33.00  43.50  40.33  40.00  40.50  31.33  49.56  47.33   
Andorra       0.0   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   

anio          2012   2013   2014   2015   2017   2018   2019  
pais                                                          
Afghanistán  37.36  37.07  37.44  37.75  39.46  37.28  36.55  
Albania      30.88  29.92  28.77  29.92  29.92  29.49  29.84  
Alemania     10.24  10.23  11.47  14.80  14.97  14.39  14.60  
Algeria      36.54  36.26  36.63  41.69  42.83  43.13  45.75  
Andorra       6.82   6.82  19.87  19.87  21.03  22.21  24.6